# Import packages and database

In [10]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.segmentation as seg
from biogeme.expressions import Beta, Variable, Derive
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [11]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [12]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [13]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [14]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [15]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [16]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [17]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

In [48]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [20]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [21]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [22]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847427,0.202567,-23.929963,0.0
asc_drive,-1.969868,0.137928,-14.281882,0.0
asc_pt,-2.337230,0.139505,-16.753691,0.0
beta_cost,-0.182006,0.016086,-11.314629,0.0
beta_time_cycle,-5.311244,0.461876,-11.499292,0.0
beta_time_drive,-6.473579,0.378946,-17.083101,0.0
beta_time_pt,-3.538116,0.250631,-14.116823,0.0
beta_time_walk,-8.430537,0.418984,-20.121406,0.0


In [23]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

In [ ]:
biogeme_model1.calculateNullLoglikelihood(av)

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [ ]:
results_specific_time.likelihood_ratio_test(results_generic, 0.05)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [24]:
gender_segmentation = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

### ASC segmentation

In [25]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(asc_drive,[gender_segmentation]).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [26]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

We can compute the result of the new model

In [27]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.455325,0.253369,-21.531171,0.000000e+00
asc_cycle_male,1.051739,0.208784,5.037450,4.717743e-07
asc_drive,-1.893902,0.144319,-13.123064,0.000000e+00
asc_drive_male,-0.198107,0.095256,-2.079723,3.755098e-02
asc_pt,-2.208746,0.146387,-15.088360,0.000000e+00
asc_pt_male,-0.333315,0.099556,-3.348020,8.139128e-04
beta_cost,-0.182830,0.016152,-11.318997,0.000000e+00
beta_time_cycle,-5.579489,0.473019,-11.795475,0.000000e+00
beta_time_drive,-6.486591,0.379978,-17.070956,0.000000e+00
beta_time_pt,-3.526173,0.251364,-14.028129,0.000000e+00


In [28]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

### Cost segmentation

And its interaction on the cost attribute in the car and public transport alternative.

In [29]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(beta_cost, [gender_segmentation]).segmented_beta()

In [30]:
v_walk_model2_2= beta_time_walk * time_walk  
v_cycle_model2_2= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2_2= asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2_2= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [31]:
V_model2_2 = {1: v_walk_model2_2 , 2: v_cycle_model2_2, 3: v_pt_model2_2, 4: v_drive_model2_2}
logprob_model2_2 = models.loglogit(V_model2_2, av, travel_mode)
biogeme_model2_2 = bio.BIOGEME(database, logprob_model2_2)
biogeme_model2_2.modelName = 'Model_2_2'
all_results['Model2_2'] = biogeme_model2_2.estimate()
results_segmented_gender_2 = biogeme_model2_2.estimate()
results_segmented_gender_2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.844027,0.202228,-23.953348,0.000000e+00
asc_drive,-1.972287,0.137775,-14.315283,0.000000e+00
asc_pt,-2.343575,0.139349,-16.818048,0.000000e+00
beta_cost,-0.151589,0.018751,-8.084372,6.661338e-16
beta_cost_male,-0.065408,0.031847,-2.053803,3.999473e-02
beta_time_cycle,-5.351123,0.459469,-11.646319,0.000000e+00
beta_time_drive,-6.485093,0.380532,-17.042162,0.000000e+00
beta_time_pt,-3.538310,0.250774,-14.109548,0.000000e+00
beta_time_walk,-8.437905,0.418584,-20.158223,0.000000e+00


In [32]:
biogeme_model2_2.calculateNullLoglikelihood(av)

-6931.471805599917

### Compiling all results from the different models

In [33]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

### Likelihood ratio test to check if model_2 and model_2_2 are equivalent or not

In [ ]:
results_segmented_gender.likelihood_ratio_test(results_segmented_gender_2, 0.05)

Model_2 is preferred because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2

### Comparison with model 1

In [54]:
results_segmented_gender.likelihood_ratio_test(results_specific_time, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=43.310838367464385, threshold=5.991464547107979)

Model_2 is preferred to Model_1 because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2.

# Model 3

In [65]:
# Power Series transformation on 'pt_dur'.

asc_cycle_3 = Beta('asc_cycle_3', 0, None, None, 0)

asc_pt_3 = Beta('asc_pt_3', 0, None, None, 0)

asc_drive_3 = Beta('asc_drive_3', 0, None, None, 0)



beta_time_walk_3 = Beta('beta_time_walk_3', 0, None, None, 0)
beta_time_cycle_3 = Beta('beta_time_cycle_3', 0, None, None, 0)

beta_time_pt_3 = Beta('beta_time_pt_3', 0, None, None, 0)
beta_time_pt_3_squarred = Beta('beta_time_pt_3_squarred', 0, None, None, 0)

beta_time_drive_3 = Beta('beta_time_drive_3', 0, None, None, 0)
beta_cost_3 = Beta('beta_cost_3', 0, None, None, 0)



v_walk_model_3= beta_time_walk_3 * time_walk 

v_cycle_model_3= asc_cycle_3 + beta_time_cycle_3 * time_cycle 

v_pt_model_3= asc_pt_3 + beta_time_pt_3 * time_pt +beta_time_pt_3_squarred * time_pt * time_pt + beta_cost * cost_pt

v_drive_model_3= asc_drive_3 + beta_time_drive_3 * time_drive + beta_cost_3 * cost_drive

In [66]:
V_model_3 = {1: v_walk_model_3 , 2: v_cycle_model_3, 3: v_pt_model_3, 4: v_drive_model_3}

logprob_model_3 = models.loglogit(V_model_3, av, travel_mode)

biogeme_model_3 = bio.BIOGEME(database, logprob_model_3)

biogeme_model_3.modelName = 'Model_3'

all_results['Model_3'] = biogeme_model_3.estimate()

results_power_3 = biogeme_model_3.estimate()

results_power_3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle_3,-4.763251,0.199840,-23.835379,0.000000e+00
asc_drive_3,-1.873665,0.139318,-13.448841,0.000000e+00
asc_pt_3,-2.601739,0.155185,-16.765448,0.000000e+00
beta_cost,-0.210762,0.031139,-6.768337,1.302713e-11
beta_cost_3,-0.168060,0.017600,-9.549035,0.000000e+00
beta_time_cycle_3,-5.431714,0.464152,-11.702455,0.000000e+00
beta_time_drive_3,-6.752512,0.442127,-15.272785,0.000000e+00
beta_time_pt_3,-2.022194,0.471842,-4.285744,1.821286e-05
beta_time_pt_3_squarred,-1.230213,0.352775,-3.487244,4.880265e-04
beta_time_walk_3,-8.319007,0.421310,-19.745558,0.000000e+00


In [67]:
#Calculate Null Log likelihood

biogeme_model_3.calculateNullLoglikelihood(av)

-6931.471805599917

In [68]:
#Comparison to model 1

results_power_3.likelihood_ratio_test(results_specific_time, 0.05)




LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=1.2207359229669237, threshold=3.841458820694124)

The model 3 is not preferred compare to model 1.

-Is it due to the non linear transformation that is not appropriate ? Should we use boxplot instead ?

-Is it stated 'non-linear transfronation on one of the variables' so I just took public transport time, should I apply to the other time ? 

-Should the non linear transformation be applied to an other time variable like car ?

-Is the loglikelyhood ratio test still appropirate ? 
